In [1]:
import mne
import numpy as np
import pandas as pd
import re

# some hyperparameter type things :)
sr = 200
t = 400
ch = 19
label = '_clean_noref'

1: obtain epochs and evokeds representations

In [ ]:
# read info from all files.
subjList = ['A_405', 'A_408', 'B_110', 'B_309', 'B_311', 'B_316', 'C_204', 'C_429', 'E_321', 'E_415', 'E_429', 'F_027', 'F_209', 'F_210', 'G_413', 'G_428', 'H_804', 'I_719', 'I_723'] # exclude 'G_413' because it only has 3 events? and the other G only has like 30 events so like wtf :(  
fileNameList = [(subj, f'preprocessing/{subj}_clean_noref.set') for subj in subjList]
fileList = [(subj, mne.io.read_raw_eeglab(fileName, verbose=False)) for subj, fileName in fileNameList]
fileList = [(subj, file, (mne.events_from_annotations(file, verbose=False))) for subj, file in fileList]
fileList = [(subj, mne.Epochs(file, events[0], event_id=[1, 2, 3, 4, 5],tmin= 0, tmax=t/sr, baseline=None, preload=True, verbose=False)) for subj, file, events in fileList]

fileListDict = dict(fileList)
fileList = []

for letter in ['A', 'B', 'C', 'E', 'F', 'G', 'H', 'I']: # 
    r = re.compile(f'{letter}_d*')
    subjects = list(filter(r.match, subjList))
    subjects = [fileListDict[x] for x in subjects]
    fileName = f'subj_{letter}'
    file = mne.concatenate_epochs(subjects)
    fileList.append((f'subj_{letter}', file, file.events[:, -1] - 1))

In [ ]:
evokeds = {subject[1][c].average() for c in ['1', '2', '3', '4', '5']}
for c in evokeds:
    c.plot_joint(title=c)

In [ ]:
for subj, file, labels in fileList:
    print(labels)

In [ ]:
# raw data as features/labels
all_features = {}
all_labels = {}

for subj, file, labels in fileList:
    features = pd.DataFrame(columns=[j for i in range(19) for j in range(401)])
    for trial_i in range(file.events.shape[0]):
        features.loc[len(features)] = file.get_data(item=trial_i).reshape(19*401) # :p i loveee hardcoding :D
    all_features[subj] = features
    all_labels[subj] = pd.Series(labels)


for subj, file, labels in fileList:
    all_features[subj].to_pickle(f'pickles/{subj}_features_clean_noref.pkl')
    all_labels[subj].to_pickle(f'pickles/{subj}_labels_clean_noref.pkl')

In [21]:
# load from .pkl files ("raw")
all_features = {}
all_labels = {}
fileList = []

for letter in ['A', 'B', 'C', 'E', 'F', 'G', 'H', 'I']: 
    all_features[f'subj_{letter}'] = np.load(f'pickles/subj_{letter}_features_raw.npy')
    all_labels[f'subj_{letter}'] = np.load(f'pickles/subj_{letter}_labels_raw.npy')
    fileList.append((f'subj_{letter}', all_features[f'subj_{letter}'], all_labels[f'subj_{letter}']))


In [ ]:
# load from .pkl files (pseudosample)
# THESE WERE NOT FIXED YET FOR LABELS -1
pseudosampled_features = {}
pseudosampled_labels = {}
fileList = []

for letter in ['A', 'B', 'C', 'E', 'F', 'G', 'H', 'I']: 
    pseudosampled_features[f'subj_{letter}'] = pd.read_pickle(f'subj_{letter}/data/pseudosampled_features_clean.pkl')
    pseudosampled_labels[f'subj_{letter}'] = pd.read_pickle(f'subj_{letter}/data/pseudosampled_labels_clean.pkl')
    fileList.append((f'subj_{letter}', all_features[f'subj_{letter}'], all_labels[f'subj_{letter}']))

In [ ]:
# pseudosampling

window = 100 # window size
strides = [75, 50, 25, 25, 50, 75, 0] # step size (but actually it's better to make it BELL CURVE OMGGGGGGG (for overlap % maybe then...))
pseudosampled_features = {}
pseudosampled_labels = {}

for subj, file, labels in fileList:
    features = pd.DataFrame(columns=[j for i in range(19) for j in range(100)])
    for trial_i in range(file.events.shape[0]):
        start = 0
        for s in strides:
            end = start + window
            features.loc[len(features)] = file.get_data(item=trial_i)[0].T[start:end].reshape(19*window)
            start += s
    pseudosampled_features[subj] = features
    pseudosampled_labels[subj] = labels

2: FFT Time !! slay

In [15]:
from scipy import fftpack
import matplotlib.pyplot as plt

def get_fft(sample): # pull one piece of that data set, trial 1 of thumb data
    sample = file.get_data(item=trial, picks=channel)[0][0]
    #fig1 = plt.plot(np.linspace(0, 1.5, num=300), sample)
    # fft on that one piece...

    sig_fft = fftpack.fft(sample)
    power = np.abs(sig_fft)**2
    sample_freq = fftpack.fftfreq(sample.size)
    return power[0:int(power.size/2)]
    # fig2 = plt.plot(sample_freq[0:200], power[0:200])

In [22]:
# do fft for everyone :)
# but acutally only try w one pereson frist ;D

sr = 200 # sample rate
t = 400 # sample size
sample_freqs = fftpack.fftfreq(t) # since all samples are the same, sample freqs are the same

fft_features = {}
fft_labels = {}

for subj, features, labels in fileList:
    # file.event_id = {'thumb':1, 'index':2, 'middle':3, 'ring':4, 'pinkie':5}
    # conditions = ['thumb']
    print(features.size)
    fft_features = pd.DataFrame(columns=[j for i in range(19) for j in sample_freqs[0:int(t/2)]])
    for trial_i in range(labels.shape[0]): #for each event
        features_flat = []
        for channel_i in range(19): #srry hardcoding this cuz lazy
            power = get_fft(features[channel_i])
            features_flat.extend(power)
        features.loc[len(features)] = features_flat
    fft_features[subj] = features
    fft_labels[subj] = labels

    print(fft_features[subj].shape)
    print(fft_labels[subj].shape)

#   print(f'{subj}: {file.events.shape[0]}')

6938800


TypeError: get_fft() missing 2 required positional arguments: 'trial' and 'channel'

STFT

In [ ]:
from scipy.signal import stft
import matplotlib.pyplot as plt

def get_stft(file, trial, channel):
    #sample = file.get_data(item=trial, picks=channel)[0][0]
    sample = file.loc[trial][channel*400:(channel+1)*400]
    f, t, Zxx = stft(sample, fs=200, window='triang', nperseg=100, return_onesided=True, boundary= None)
    Zxx = np.abs(Zxx)
    print(Zxx.shape)
    # plt.pcolormesh([t], f, np.abs(Zxx), vmin=Zxx.min(), vmax=Zxx.max(), shading='gourand')
    # plt.title('STFT Magnitude')
    # plt.ylabel('Frequency [Hz]')
    # plt.xlabel('Time [sec]')
    # plt.show()
    return f, t, Zxx

file = all_features['subj_A']
STFT = get_stft(file, 1, 1)

In [ ]:
fileList = fileList[0:1]
STFT_features = {}
STFT_labels = {}

for subj, file, labels in fileList:
    features = pd.DataFrame(columns=[j for j in range(387) for i in range(19)])
    for trial_i in range(labels.shape[0]): #for each event
        features_flat = []
        for channel_i in range(19): #srry hardcoding this cuz lazy
            sample = file.loc[trial_i][channel_i*400:(channel_i+1)*400]
            f, t, Zxx = stft(sample, fs=200, window='triang', boundary= None)
            features_flat.extend(np.reshape(Zxx, (Zxx.shape[0]*Zxx.shape[1])))
        features.loc[len(features)] = features_flat
    # features.to_csv(f'{subj}/fft/features.csv', index=False)
    # labels.to_csv(f'{subj}/fft/labels.csv', index=False)
    STFT_features[subj] = features
    STFT_labels[subj] = labels

1D CNN:

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split

class ConvNet_1D(nn.Module):
    def __init__(self):
        super(ConvNet_1D, self).__init__()
        ### FILL IN ### [10 POINTS]

        self.conv1 = nn.Sequential(
            nn.Conv1d(19, 32, 3),
            nn.ReLU(), 
            nn.MaxPool1d(2)
        )
        
        self.conv2 = nn.Sequential(
            nn.Conv1d(32, 64, 3),
            nn.ReLU(),
            nn.MaxPool1d(2)
        )

        self.conv3 = nn.Sequential(
            nn.Conv1d(32, 64, 7),
            nn.ReLU(),
            nn.MaxPool1d(2)
        )

        self.hidden = nn.Sequential(
            nn.Flatten(),
            # nn.Linear(121728, 256),
            nn.Linear(6080, 256), # all features norefer
            # nn.Linear(1408, 256), #pseudosampled featrues .
            nn.Linear(256, 5)  
        )

    def forward(self, x):
        ### FILL IN ### [5 POINTS]
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.hidden(x)
        return x

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
# train neural net
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

for subj, features, labels in fileList:
    features = STFT_features[subj]
    labels = STFT_labels[subj]
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size= 0.3) # 70% training 30% test
    conv_net = ConvNet_1D().to(device)
    criterion = nn.CrossEntropyLoss()
    conv_net_optimizer = torch.optim.Adam(conv_net.parameters(), lr=0.001)
    # Train the neural network
    for epoch in range(10):
        print("epoch = ",epoch)
        conv_net.train()
        data = X_train.to_numpy()
        targets = y_train.to_numpy()
        data = torch.tensor(data.reshape(int(data.shape[0]), 19, int(data.shape[1]/19)), dtype=torch.float32) # reshape # of trial, 1 channel, # of samples
        data = data.to(device)
        targets = torch.tensor(targets, dtype=torch.int64)
        targets = targets.to(device)

        # Forward pass
        conv_net_predictions = conv_net(data)
        conv_net_loss = criterion(conv_net_predictions, targets)

        # Backward pass
        conv_net_optimizer.zero_grad()
        conv_net_loss.backward()
        conv_net_optimizer.step()

        # Evaluate the neural network
        conv_net.eval()
        correct, total = 0, 0
        with torch.no_grad():
            data = X_test.to_numpy()
            targets = y_test.to_numpy()
            data = torch.tensor(data.reshape(int(data.shape[0]), 19, int(data.shape[1]/19)), dtype=torch.float32) # reshape # of trial, 1 channel, # of samples
            data = data.to(device)
            targets = torch.tensor(targets, dtype=torch.int64)
            targets = targets.to(device)
            conv_net_predictions = conv_net(data)
            _, predicted = torch.max(conv_net_predictions.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
        print("test accuracy = ",correct/total)



2D CNN

In [ ]:
class eeg(Dataset):
    def __init__(self, dataframe):
    
    def __len__(self):
        return len(self.dataframe)
    
    def __getitem__(self, idc):
        if torch.is_tensor(idx):
            idx = idx.tolist()

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split

class ConvNet_2D(nn.Module):
    def __init__(self):
        super(ConvNet_2D, self).__init__()
        ### FILL IN ### [10 POINTS]

        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 8, (7, 3)),
            nn.ReLU(), 
            #nn.MaxPool2d(2)
        )
        
        self.conv2 = nn.Sequential(
            nn.Conv2d(8, 16, (5, 3)),
            nn.Conv2d(16, 32, (3, 3)),
            nn.ReLU(),
            #nn.MaxPool2d(2)
        )

        self.conv3 = nn.Sequential(
            nn.ReLU(),
            #nn.MaxPool2d(2)
        )

        self.hidden = nn.Sequential(
            nn.Flatten(),
            # nn.Linear(121728, 256),
            nn.Linear(43456, 256),
            nn.ReLU(),
            nn.Dropout(0.4), # all features norefer
            # nn.Linear(1408, 256), #pseudosampled featrues .
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(128, 5),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        ### FILL IN ### [5 POINTS]
        x = self.conv1(x)
        x = self.conv2(x)
        # x = self.conv3(x)
        x = self.hidden(x)
        return x

In [ ]:
# binary class ?:D
for subj, features, labels in fileList[0:1]:
    joint = features.assign(label=labels.values)
    joint =  joint.loc[joint['label'].isin([0, 4])]
    features = joint.drop(['label'], axis=1)
    labels = joint['label']

    all_features[subj] = features
    all_labels[subj] = labels

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
# train neural net
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

for subj, features, labels in fileList:
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size= 0.3) # 70% training 30% test
    conv_net = ConvNet_2D().to(device)
    criterion = nn.CrossEntropyLoss()
    conv_net_optimizer = torch.optim.Adam(conv_net.parameters(), lr=0.001)
    # Train the neural network
    for epoch in range(100):
        #print("epoch = ",epoch)
        conv_net.train()
        data = X_train.to_numpy()
        targets = np.subtract(y_train.to_numpy(), 1)
        data = torch.tensor(data.reshape(int(data.shape[0]), 1, 19, int(data.shape[1]/19)), dtype=torch.float32) # reshape # of trial, 1 channel, # of samples
        data = data.to(device)
        targets = torch.tensor(targets, dtype=torch.int64)
        targets = targets.to(device)

        # Forward pass
        conv_net_predictions = conv_net(data)
        conv_net_loss = criterion(conv_net_predictions, targets)

        # Backward pass
        conv_net_optimizer.zero_grad()
        conv_net_loss.backward()
        conv_net_optimizer.step()

        # Evaluate the neural network
        conv_net.eval()
        correct, total = 0, 0
        with torch.no_grad():
            data = X_test.to_numpy()
            targets = np.subtract(y_test.to_numpy(), 1)
            data = torch.tensor(data.reshape(int(data.shape[0]), 1, 19, int(data.shape[1]/19)), dtype=torch.float32) # reshape # of trial, 1 channel, # of samples
            data = data.to(device)
            targets = torch.tensor(targets, dtype=torch.int64)
            targets = targets.to(device)
            conv_net_predictions = conv_net(data)
            _, predicted = torch.max(conv_net_predictions.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
    print("test accuracy = ",correct/total)

test accuracy =  0.3333333333333333
test accuracy =  0.2
test accuracy =  0.2
test accuracy =  0.4
test accuracy =  0.13333333333333333
test accuracy =  0.26666666666666666
test accuracy =  0.2
test accuracy =  0.4666666666666667


CNN w/ channels

3: SVM

In [8]:
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import metrics

In [10]:
# SVM TIME WOOO
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import metrics

for subj, features, labels in fileList:
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size= 0.3) # 70% training 30% test
    
    classifier = svm.SVC(kernel='rbf') # radial basis kernel; gamma should be 0.1
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)

    print(f"Accuracy {subj}:",metrics.accuracy_score(y_test, y_pred))

Accuracy subj_A: 0.2
Accuracy subj_B: 0.13333333333333333
Accuracy subj_C: 0.06666666666666667
Accuracy subj_E: 0.26666666666666666
Accuracy subj_F: 0.3333333333333333
Accuracy subj_G: 0.3333333333333333
Accuracy subj_H: 0.2
Accuracy subj_I: 0.3333333333333333


In [ ]:
# SVM TIME WOOO
# SVM w/ 2 seconds of data takes much more time and does not improve accuracy :(
# w/ 0.85 seconds it works .2 % maybe better and takes 1/3 of the time

avg_acc = 0
for subj, features, labels in fileList:
    features = STFT_features[subj]
    labels = STFT_labels[subj]
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size= 0.3) # 70% training 30% test
    classifier = svm.SVC(kernel='rbf') # radial basis kernel; gamma should be 0.1
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    avg_acc += metrics.accuracy_score(y_test, y_pred)

    print(f"Accuracy {subj}:", metrics.accuracy_score(y_test, y_pred))
print("Average accuracy: ", avg_acc/len(fileList))

random forest ? !

In [ ]:
# Import the model we are using
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics

for subj, features, labels in fileList:
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size= 0.3) # 70% training 30% test

    rf = RandomForestClassifier(n_estimators = 100)
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)

    print(f"Accuracy {subj}:",metrics.accuracy_score(y_test, y_pred))

In [ ]:

for subj, file in fileList:
    features = pseudosampled_features[subj]
    labels = pseudosampled_labels[subj]
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size= 0.3) # 70% training 30% test

    rf = RandomForestClassifier(n_estimators = 100)
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)

    print(f"Accuracy {subj}:",metrics.accuracy_score(y_test, y_pred))

3: try morlet??

In [ ]:
print(tfr.data.shape)
freqs.shape

In [ ]:
# Define frequencies of interest
freqs = np.logspace(*np.log10([6, 30]), num=20) 

for subj, file in fileList:
    tfr = mne.time_frequency.tfr_morlet(file, freqs=freqs, n_cycles=freqs / 2., return_itc=False)
    features = tfr.data.reshape((tfr.data.shape[0]*tfr.data.shape[1]* tfr.data.shape[2]))
    labels = [file.events[trial][2] for trial in file.events.shape[0]]  # Extract labels from the epochs


In [ ]:
# get CSP features :D
# use raw data frames not mne.epochs datastructure

from mne.decoding import CSP

csp_features = {}
csp_labels = {}

for subj, file, labels in fileList:
    csp = CSP(n_components=5, reg=None, log=True)
    data = np.reshape(all_features[subj].to_numpy(), (all_features[subj].shape[0], 19, 401))
    labels = all_labels[subj].to_numpy()

    csp_features[subj] = csp.fit_transform(data, labels)
    csp_labels[subj] = labels

    csp.plot_filters(file.info, title='CSP patterns for %s' % subj)    

